In [2]:
import pandas as pd
import numpy as np
from readFile import split_into_values, toRPdata

In [3]:
# columns 와 value는 사용자 입력
#df = pd.read_csv('resources/AXISX_resample.csv')
df = pd.read_csv('resources/CLAMP_resample.csv')
value = ['value']
#df = pd.read_csv('resources/Dataset1.csv')
columns = ['chip', 'wire', 'segment']
value = ['value']

In [4]:
df = df.loc[:, columns + value] #('chip', 'wire', 'value')는 사용자 입력
size = 28 # 모델에 따라 변경 28, 96
result = split_into_values(df, columns)

In [5]:
# dataframe -> list
result_list = result.values.tolist()

In [6]:
from sklearn.preprocessing import MinMaxScaler

def MinMax(data):
    MMS = MinMaxScaler().fit(data)
    scaled = MMS.transform(data)
    return scaled

In [7]:
# result_list transpose
result_T = [list(x) for x in zip(*result_list)]

In [8]:
# minmax 정규화
result_scaled = MinMax(result_T)

In [9]:
# 다시 result transpose 해서 원래대로
result_scaled = [list(x) for x in zip(*result_scaled)]

In [10]:
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

In [11]:
# 2. 시계열 셋 크기 변경
result_ = TimeSeriesResampler(sz=size).fit_transform(result)

In [12]:
data = result_.reshape(result_.shape[0], 1, size)
X = toRPdata(data, threshold='point', percentage=30) # rp 1
#X = toRPdata(data) # rp 2

In [13]:
X_scaled = np.expand_dims(X, axis=3)
X_scaled.shape

(1140, 28, 28, 1)

In [14]:
from keras.models import load_model
model = load_model('CLAMP28_weight.h5')

In [15]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
activation_10 (Activation)   (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 2)         290       
_________________________________________________________________
activation_11 (Activation)   (None, 14, 14, 2)         0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 2)           0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 7, 7, 2)          

In [16]:
from keras import backend as K

compressed_layer = 5
get_layer_output = K.function([model.layers[0].input],[model.layers[compressed_layer].output])
compressed = get_layer_output([X_scaled])[0]
feature = compressed.reshape(compressed.shape[0], compressed.shape[1]*compressed.shape[2]*compressed.shape[3])

print(feature)
print(feature.shape)

[[0.88457197 0.         0.10616754 ... 0.15150811 1.4553685  0.31374487]
 [0.6336747  0.         0.05666872 ... 0.5718753  1.1884595  0.0213138 ]
 [0.74471104 0.         0.34963053 ... 0.2087781  0.88176906 0.1399442 ]
 ...
 [0.6336747  0.         0.04583086 ... 0.5718753  1.1884595  0.0213138 ]
 [0.84329295 0.         1.0842481  ... 0.1652462  1.1471374  0.0213138 ]
 [0.74471104 0.         0.3369028  ... 0.15290315 1.6502179  0.18481062]]
(1140, 98)


In [17]:
import numpy

numpy.save('./compressed3', compressed) # x_save.npy
numpy.save('./feature3', feature) # x_save.npy
numpy.save('./X_scaled3', X_scaled) # x_save.npy